<a href="https://colab.research.google.com/github/nakamura196/ndl_ocr/blob/main/Archivematica%E3%81%AEAPI%E3%82%92%E4%BD%BF%E3%81%A3%E3%81%A6%E3%81%BF%E3%82%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ArchivematicaのAPIを使ってみる

In [ ]:
import requests
import base64
import time
from tqdm import tqdm

# サーバの設定

In [ ]:
hostname = "matica.aws.ldas.jp"
endpoint = f"http://{hostname}:81/api"
username = "demo"
api_key = "b9f20c2a99c0e2d91c53061b2834a1b68a5f8b7b"
location_uuid = "a71d85ac-aeb0-4258-a931-688d5a84a87b"

In [ ]:
headers = {'Authorization': f'ApiKey {username}:{api_key}'}

# Transferの設定

In [ ]:
name = "mc_api_transfer"
type = "standard"
accession = "2023-1234"
paths = ["files/jpg"]
row_ids = [""]

## base64へエンコード

In [ ]:
paths_encoded = []

for path in paths:
  path_encoded = base64.b64encode(f"{location_uuid}:{path}".encode()).decode()
  paths_encoded.append(path_encoded)

paths_encoded

In [ ]:
data = {
    "name": name,
    "type": type,
    "accession": accession,
    "paths[]": paths_encoded,
    "row_ids[]": row_ids
}

response = requests.post(f'{endpoint}/transfer/start_transfer/', headers=headers, data=data)

response

In [ ]:
response.json()

## （参考） List unapproved transfer(s)


In [ ]:
response_2 = requests.get(f'{endpoint}/transfer/unapproved/', headers=headers)
response_2.json()

### 複数のtransferを一括でapproveする



In [ ]:
results = response_2.json()["results"]
for result in tqdm(results):

  data = {
      "type": result["type"],
      "directory": result["directory"]
  }

  requests.post(f'{endpoint}/transfer/approve/', headers=headers, data=data)

## Approve transfer(s)

In [ ]:
directory = response.json()["path"].split("/")[-2]
directory

In [ ]:
while 1:

  data = {
      "type": type,
      "directory": directory
  }

  response3 = requests.post(f'{endpoint}/transfer/approve/', headers=headers, data=data)

  response3.json()

  if "uuid" in response3.json():
    break

  time.sleep(5)

In [ ]:
transfer_UUID = response3.json()["uuid"]
transfer_UUID

## transferのstatus

In [ ]:
while 1:

  response5 = requests.get(f'{endpoint}/transfer/status/{transfer_UUID}', headers=headers)

  response5.json()

  if response5.json()["status"] != "PROCESSING":
    break

  time.sleep(5)

In [ ]:
sip_uuid = response5.json()["sip_uuid"]
sip_uuid

## （参考） completed

In [ ]:
response_completed = requests.get(f'{endpoint}/transfer/completed/', headers=headers)
response_completed.json()

In [ ]:
# completedを削除
results = response_completed.json()["results"]
for result in tqdm(results):
  requests.delete(f'{endpoint}/transfer/{result}/delete', headers=headers)

### 削除



In [ ]:
transfer_UUID = "0a6699b8-be29-4d6b-90ee-70c471ebfe8f"
response5 = requests.get(f'{endpoint}/transfer/status/{transfer_UUID}', headers=headers)
response5.json()

In [ ]:
response_completed = requests.delete(f'{endpoint}/transfer/{transfer_UUID}/delete', headers=headers)
print(response_completed)

# Ingest

処理の進捗をwhileで確認する

In [ ]:
while 1:

  response4 = requests.get(f'{endpoint}/ingest/status/{sip_uuid}', headers=headers)

  if response4.json()["status"] != "PROCESSING":
    break

  time.sleep(5)

## 入力待ちの一覧

In [ ]:
response4 = requests.get(f'{endpoint}/ingest/waiting', headers=headers)
response4.json()

In [ ]:
sip_uuid = "77acfa8d-d212-42e1-a773-a37c7697e89a"
response_d = requests.delete(f'{endpoint}/ingest/{sip_uuid}/delete/', headers=headers)
response_d.text

## 完了済みを削除

In [ ]:
completed = requests.get(f'{endpoint}/ingest/completed/', headers=headers)
results = completed.json()["results"]
for result in tqdm(results):
  requests.delete(f'{endpoint}/ingest/{result}/delete/', headers=headers)

# AIPのダウンロード

In [ ]:
from requests.auth import HTTPBasicAuth

uuid = response4.json()["uuid"]

username="demo"
password="Nd4Ev3XJ"

endpoint2 = f"http://{hostname}:8001/api"

url = f"{endpoint2}/v2/file/{uuid}?format=json"

# print(url)

response6 = requests.get(url, headers=headers, auth=HTTPBasicAuth(username, password))

response6.json()

In [ ]:
url2 = f"{endpoint2}/v2/file/{uuid}/download/"

response7 = requests.get(url2, headers=headers, auth=HTTPBasicAuth(username, password))

content = response7.content

output_path = "test.7z"

with open(output_path ,mode='wb') as f: # wb でバイト型を書き込める
  f.write(content)

In [ ]:
from google.colab import files
files.download(output_path)